In [ ]:
import json
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import gamma
from scipy.special import gamma as gamma_fn
import re
import os
from matplotlib import pyplot as plt
from typing import Tuple, List, Callable, Any

from distutils.dir_util import copy_tree
from run_config_json import create_run_config_json

In [ ]:
# General settings
num_runs = 1
starting_seed = 0
seed_multiplier = 100

# Validator settings
num_nodes = 16 
num_consensus = 2000
base_time_limit = 10000
node_processing_distribution = "exp"
node_processing_parameters = [3]
consensus_protocol = "IBFT"

## Fault settings
num_faults = 1
fault_type = "UR"
fault_parameters = []

# Network settings
## Switch settings
switch_processing_distribution = "degen"
switch_processing_parameters = [0]
message_channel_success_rate = 1

network_type = "Clique"
network_parameters = []

In [ ]:
VALIDATOR_RESULTS_FILEPATH = "../json"
RESULTS_DIRECTORY = "json_n{num_nodes}_btl{base_time_limit:.1f}_{node_dist}_{node_params}_{topology}_{topo_params}_{switch_dist}_{switch_params}" +  \
                    "_{protocol}_{num_faults}_{fault_type}_{fault_params}"

In [ ]:
def write_str_to_file(file_string: str, filename: str) -> None:
    with open(filename, "w") as file:
        file.write(file_string)

def run_and_save(run_config_dic: str, output_directory: str) -> None:
    config_filename = "config.json"
    write_str_to_file(run_config_dic, config_filename)

    argument = "py/" + config_filename
    ! (cd "../" && gradlew run --args={argument})
    copy_tree(VALIDATOR_RESULTS_FILEPATH, output_directory)    

def construct_results_directory(num_nodes: int, base_time_limit: float, 
                                node_processing_distribution: str, node_processing_parameters: List[float],
                                topology: str, topo_params: List[int], switch_processing_distribution: str, switch_processing_parameters: List[float], 
                                protocol: str, num_faults: int, fault_type: str, fault_params: List[int]) -> str:
    return RESULTS_DIRECTORY.format(num_nodes=num_nodes,  base_time_limit=base_time_limit, 
                                    node_dist=node_processing_distribution, node_params=node_processing_parameters, 
                                    topology=topology, topo_params=topo_params, switch_dist=switch_processing_distribution, switch_params=switch_processing_parameters, 
                                    protocol=protocol, num_faults=num_faults, fault_type=fault_type, fault_params=fault_params)


In [ ]:
results_dic = {}
for num_faults in [4]:
    # for base_time_limit in [10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20, 20.5, 21, 21.5, 22, 22.5, 23, 24, 25, 26, 27, 28, 29, 30]: 
    for base_time_limit in [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30] + list(range(31, 51, 2)): 
# for base_time_limit in range(31, 51):
    # for base_time_limit in [5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5]:
# for base_time_limit in [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5]:
# for num_nodes in [4, 8, 16, 24, 32, 48, 64]:
        json_obj = create_run_config_json(num_runs, starting_seed, seed_multiplier,
                                      num_nodes, num_consensus, base_time_limit, 
                                      node_processing_distribution, node_processing_parameters, 
                                      consensus_protocol, num_faults, fault_type, fault_parameters,
                                      switch_processing_distribution, switch_processing_parameters, 
                                      message_channel_success_rate, network_type, network_parameters)
        run_and_save(json_obj, construct_results_directory(num_nodes, float(base_time_limit), 
                                                       node_processing_distribution, node_processing_parameters, 
                                                       network_type.lower(), network_parameters, 
                                                       switch_processing_distribution, switch_processing_parameters, 
                                                       consensus_protocol.lower(),
                                                       num_faults, fault_type, fault_parameters))
